In [1]:
import numpy as np
import tensorflow as tf
import math
from tensorflow import keras
import numpy as np

2025-03-13 11:08:39.032653: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741846119.142767   16381 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741846119.173308   16381 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-13 11:08:39.432887: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
texts = ["hi, how are you doing?", "i'm fine. how about yourself?", "i'm fine. how about yourself?", "i'm pretty good. thanks for asking.",
       "i'm pretty good. thanks for asking.", "no problem. so how have you been?", "no problem. so how have you been?", 
       "i've been great. what about you?", "i've been great. what about you?"]

In [3]:
texts

['hi, how are you doing?',
 "i'm fine. how about yourself?",
 "i'm fine. how about yourself?",
 "i'm pretty good. thanks for asking.",
 "i'm pretty good. thanks for asking.",
 'no problem. so how have you been?',
 'no problem. so how have you been?',
 "i've been great. what about you?",
 "i've been great. what about you?"]

In [6]:
tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)


In [7]:
padded_sequences = keras.preprocessing.sequence.pad_sequences(sequences, padding='post')

In [81]:
maxlen=100
dmodel=16
num_heads=4
dff = 1024
num_layers = 4
vocab_size = 100

In [9]:
inputs = tf.convert_to_tensor(padded_sequences, dtype=tf.float32)
embedding_layer = keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=dmodel, input_length=maxlen)
inputs = embedding_layer(inputs)

2025-03-13 11:08:59.255355: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/greenfin/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [10]:
print(texts)
print(inputs)

['hi, how are you doing?', "i'm fine. how about yourself?", "i'm fine. how about yourself?", "i'm pretty good. thanks for asking.", "i'm pretty good. thanks for asking.", 'no problem. so how have you been?', 'no problem. so how have you been?', "i've been great. what about you?", "i've been great. what about you?"]
tf.Tensor(
[[[-0.01098777 -0.01075876  0.01202124 ...  0.02710631 -0.03660393
   -0.02707372]
  [-0.00424042 -0.04119848 -0.01805595 ...  0.04775662  0.02265276
    0.04632482]
  [-0.04126381 -0.00921334  0.03514783 ... -0.04007578 -0.00032945
    0.03011623]
  ...
  [-0.04419166 -0.03896519 -0.02903279 ... -0.02920136 -0.03805124
    0.02695042]
  [ 0.01419434  0.04794849  0.02349133 ... -0.04870173  0.00471635
    0.00758206]
  [ 0.01419434  0.04794849  0.02349133 ... -0.04870173  0.00471635
    0.00758206]]

 [[-0.02594265 -0.04670627  0.013793   ... -0.00706627 -0.01871919
   -0.00097572]
  [ 0.02264768  0.02398337 -0.04740096 ... -0.03811659  0.03920604
    0.00150979]


In [11]:
class PositionalEncoding(keras.layers.Layer):
    def __init__(self, maxlen, dmodel):
        super(PositionalEncoding, self).__init__()
        #Dimension of the vector
        self.dmodel = dmodel
        # Positions, is the columns vectore that represent the position of the token in sequence from "zero" to "maxlen"
        # Position vevtor hold the position of each token in sequence (Position indes of each token)
        positions = np.arange(maxlen)[:, np.newaxis]
        # index are row vector represent the dimension indices of positional encoding
        index = np.arange(dmodel)[np.newaxis, :]
        #angles Rates, The scaling factors applied to each dimension based on the positional index. 
        # These scale the position values differently for each dimension
        angle_rates = 1/np.power(10000, (2 * (index // 2)) / np.float32(dmodel))
        # Angle_rads, is scaling factors that applied to each dimension based on the positional index. 
        angle_rads = positions*angle_rates
        # Apply sign on even indices
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
        # Apply cos on odd indices
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
        # ready the positional embiddibg for input and change the type to tensorflow f.32
        self.positional_encoding = tf.cast(angle_rads[np.newaxis, ...], dtype=tf.float32)
    def call(self, inputs):
        return inputs + self.positional_encoding[:, :tf.shape(inputs)[1], :]

In [17]:
pos_encoding_layer = PositionalEncoding(maxlen=maxlen, dmodel=dmodel)
Pos_out = pos_encoding_layer(inputs)
print(Pos_out.shape)


(9, 7, 16)


In [18]:
Pos_out

<tf.Tensor: shape=(9, 7, 16), dtype=float32, numpy=
array([[[-1.0987770e-02,  9.8924124e-01,  1.2021244e-02, ...,
          1.0271063e+00, -3.6603928e-02,  9.7292626e-01],
        [ 8.3723056e-01,  4.9910378e-01,  2.9292765e-01, ...,
          1.0477562e+00,  2.2968983e-02,  1.0463247e+00],
        [ 8.6803359e-01, -4.2536017e-01,  6.2627494e-01, ...,
          9.5992219e-01,  3.0300947e-04,  1.0301161e+00],
        ...,
        [-8.0099416e-01, -6.9260877e-01,  9.2454797e-01, ...,
          9.7079062e-01, -3.6786325e-02,  1.0269496e+00],
        [-9.4472998e-01,  3.3161068e-01,  1.0234379e+00, ...,
          9.5128578e-01,  6.2974859e-03,  1.0075808e+00],
        [-2.6522115e-01,  1.0081187e+00,  9.7063947e-01, ...,
          9.5128030e-01,  6.6137132e-03,  1.0075803e+00]],

       [[-2.5942648e-02,  9.5329374e-01,  1.3793003e-02, ...,
          9.9293375e-01, -1.8719185e-02,  9.9902427e-01],
        [ 8.6411864e-01,  5.6428564e-01,  2.6358265e-01, ...,
          9.6188295e-01,  3.952

In [19]:
class ScaleDotProductAttention(keras.layers.Layer):
    def __init__(self):
        super(ScaleDotProductAttention, self).__init__()
        
    def call(self, Query, Key, Value, Mask=None):
        matmul_QK = tf.matmul(Query,Key, transpose_b=True)
        Depth = tf.cast(tf.shape(Key)[-1], tf.float32)
        logits = matmul_QK / tf.math.sqrt(Depth)
        
        if Mask is not None:
            logits += (Mask * -1e9)
        Attention_weights = tf.nn.softmax(logits, axis=-1)
        Output = tf.matmul(Attention_weights,Value)
        return Output, Attention_weights

In [20]:
dot_prod = ScaleDotProductAttention()

In [21]:
dot_out, weights = dot_prod(Pos_out,Pos_out,Pos_out)

In [22]:
dot_out

<tf.Tensor: shape=(9, 7, 16), dtype=float32, numpy=
array([[[ 3.15977558e-02,  2.25514054e-01,  5.95164418e-01, ...,
          9.89263058e-01, -3.11518251e-03,  1.00873506e+00],
        [ 1.10555522e-01,  1.44054443e-01,  5.97265005e-01, ...,
          9.89154398e-01, -1.09454244e-03,  1.01044047e+00],
        [ 9.78088081e-02,  3.34167480e-03,  6.41411543e-01, ...,
          9.84834492e-01, -2.35557731e-04,  1.01036024e+00],
        ...,
        [-1.25122100e-01, -3.15713435e-02,  7.26874113e-01, ...,
          9.77756321e-01, -2.95801391e-03,  1.00864363e+00],
        [-1.63964778e-01,  1.19323589e-01,  7.24589765e-01, ...,
          9.77805912e-01, -3.07695242e-03,  1.00865197e+00],
        [-1.09756529e-01,  2.18234867e-01,  6.98978066e-01, ...,
          9.79841352e-01, -2.03885324e-03,  1.00903118e+00]],

       [[ 4.63533215e-02,  2.25198388e-01,  5.97640991e-01, ...,
          9.88369823e-01,  6.10709842e-03,  1.00890541e+00],
        [ 1.34744823e-01,  1.56781927e-01,  5.89952

In [23]:
weights

<tf.Tensor: shape=(9, 7, 7), dtype=float32, numpy=
array([[[0.20071319, 0.17856538, 0.13256925, 0.10816687, 0.11130411,
         0.12938271, 0.13929857],
        [0.16848323, 0.19587204, 0.16873464, 0.13009712, 0.10636107,
         0.10721573, 0.12323606],
        [0.12550813, 0.1693066 , 0.18925534, 0.17082998, 0.13140832,
         0.10689902, 0.1067927 ],
        [0.10262579, 0.1308189 , 0.17119744, 0.19602658, 0.17249702,
         0.12535866, 0.10147559],
        [0.10369342, 0.10501786, 0.12931047, 0.16937892, 0.19936879,
         0.16803893, 0.1251916 ],
        [0.12266173, 0.10772883, 0.10704774, 0.12526362, 0.17100263,
         0.19417648, 0.17211899],
        [0.13740687, 0.12883689, 0.11126904, 0.1055021 , 0.1325553 ,
         0.17908444, 0.20534535]],

       [[0.19807506, 0.1741881 , 0.13580763, 0.111372  , 0.11198395,
         0.12989229, 0.13868098],
        [0.17118044, 0.19476618, 0.17236726, 0.1297763 , 0.1035323 ,
         0.10555797, 0.12281945],
        [0.12717983,

In [24]:
class MultiHeadAttention(keras.layers.Layer):
    def __init__(self, num_heads, dmodel):
        super(MultiHeadAttention, self).__init__()
        self.dmodel = dmodel
        self.num_heads = num_heads
        #
        assert dmodel % self.num_heads == 0
        self.Depth = dmodel // self.num_heads
        #
        self.Query_Den = keras.layers.Dense(units=dmodel)
        self.Key_Den = keras.layers.Dense(units=dmodel)
        self.Val_Den = keras.layers.Dense(units=dmodel)
        #
        self.attention = ScaleDotProductAttention()
        #
        self.output_Den = keras.layers.Dense(units=dmodel)
        #
    def SplitHeads(self, X, Batch_Size):
        x = tf.reshape(X, shape=(Batch_Size, -1,self.num_heads, self.Depth))
        return tf.transpose(x, perm=[0,2,1,3])
        
    def call(self, Query, Key, Value, Mask=None):
        batch_size = tf.shape(Query)[0]
        # Dense layer on all of inputs
        query = self.Query_Den(Query)
        key = self.Key_Den(Key)
        value = self.Val_Den(Value)
        # splits "Query", "Key" and "Value" into multiple heads
        query = self.SplitHeads(query, batch_size)
        key = self.SplitHeads(key, batch_size)
        value = self.SplitHeads(value,batch_size)
        #
        Output, Weights = self.attention(query,key,value,Mask)
        #
        Output = tf.transpose(Output,perm=[0,2,1,3])
        Output = tf.reshape(Output,(batch_size, -1, self.dmodel))
        #
        Output = self.output_Den(Output)
        #
        return Output, Weights

In [25]:
multi_head_attention = MultiHeadAttention(num_heads=num_heads, dmodel=dmodel)

In [26]:
Attn_out, attention_weights = multi_head_attention(Pos_out, Pos_out, Pos_out)

In [27]:
Attn_out

<tf.Tensor: shape=(9, 7, 16), dtype=float32, numpy=
array([[[ 0.1359101 , -0.67935777,  0.97902673, ...,  0.81126314,
         -1.2058667 , -0.49916032],
        [ 0.11140864, -0.6750976 ,  0.96505606, ...,  0.8122632 ,
         -1.1894518 , -0.4966038 ],
        [ 0.11782763, -0.65924853,  0.95400214, ...,  0.8130834 ,
         -1.1808038 , -0.5137639 ],
        ...,
        [ 0.17227322, -0.64310265,  0.97641706, ...,  0.81550646,
         -1.2068074 , -0.5603053 ],
        [ 0.17475423, -0.65890384,  0.99708396, ...,  0.8123693 ,
         -1.2277603 , -0.56056786],
        [ 0.14615606, -0.6734803 ,  1.0058612 , ...,  0.81141955,
         -1.2276975 , -0.5561477 ]],

       [[ 0.12358905, -0.6683887 ,  0.97006106, ...,  0.8267393 ,
         -1.1907276 , -0.5068213 ],
        [ 0.10025668, -0.6656101 ,  0.95269936, ...,  0.8256761 ,
         -1.1745619 , -0.4977182 ],
        [ 0.10206643, -0.6464789 ,  0.94188535, ...,  0.8306124 ,
         -1.1590053 , -0.52101076],
        ...,
  

In [28]:
attention_weights

<tf.Tensor: shape=(9, 4, 7, 7), dtype=float32, numpy=
array([[[[0.1419315 , 0.11032497, 0.09260158, ..., 0.1490704 ,
          0.20070508, 0.20487861],
         [0.1491628 , 0.1138794 , 0.09657454, ..., 0.15012763,
          0.19494796, 0.19069152],
         [0.17356792, 0.12680309, 0.1017407 , ..., 0.13758314,
          0.18272474, 0.1768083 ],
         ...,
         [0.19111322, 0.1367077 , 0.09951928, ..., 0.12034659,
          0.17932822, 0.18587907],
         [0.17270736, 0.13214442, 0.09890087, ..., 0.12334095,
          0.1841789 , 0.20046024],
         [0.15434664, 0.12710191, 0.10254899, ..., 0.13277753,
          0.1852648 , 0.200856  ]],

        [[0.14676209, 0.15234102, 0.1539719 , ..., 0.13817446,
          0.12916678, 0.12855604],
         [0.14894447, 0.16157109, 0.16706637, ..., 0.13450257,
          0.11456487, 0.11073431],
         [0.14227669, 0.15650417, 0.17232257, ..., 0.14281645,
          0.11037407, 0.09877435],
         ...,
         [0.12237208, 0.12591885, 

In [29]:
class AddNorm(keras.layers.Layer):
    def __init__(self, Epsilon=1e-6):
        super(AddNorm,self).__init__()
        self.layer_norm = keras.layers.LayerNormalization(epsilon=Epsilon)
    def call(self, Attn_out, Pos_out):
        return self.layer_norm(Attn_out+Pos_out)

In [30]:
Norm = AddNorm()
Norm1_out = Norm(Attn_out,Pos_out)

In [31]:
class FeedForwardNetwork(keras.layers.Layer):
    def __init__(self, dmodel, dff):
        super(FeedForwardNetwork, self).__init__()
        self.dff = dff
        #
        self.Dense1 = keras.layers.Dense(units=self.dff, activation='relu')
        #self.Dense2 = keras.layers.Dense(units=self.dff//2, activation='relu')
        self.out = keras.layers.Dense(units=dmodel)
    def call(self, norm_out):
        return self.out(self.Dense1(norm_out))

In [32]:
fedfwd = FeedForwardNetwork(dmodel=dmodel, dff=dff)

In [33]:
feedfrwd_out = fedfwd(Norm1_out)

In [34]:
class EncoderLayer(keras.layers.Layer):
    def __init__(self, dmodel, dff, num_heads, dropout_rate = 0.1):
        super(EncoderLayer, self).__init__()
        
        self.MHA = MultiHeadAttention(num_heads=num_heads, dmodel=dmodel)
        self.Norm1 = AddNorm()
        self.Norm2 = AddNorm()
        self.feedforward = FeedForwardNetwork(dmodel=dmodel, dff=dff)
        self.Dropout1 = keras.layers.Dropout(dropout_rate)
        self.Dropout2 = keras.layers.Dropout(dropout_rate)
        #
    def call(self, Pos_Out, Mask=None):
        #
        Attn_output, _ = self.MHA(Pos_Out, Pos_Out, Pos_Out, Mask)   # In Attention, our Input are feeded as three inputs 
        #                                                      #            
        Attn_output = self.Dropout1(Attn_output)               #            Dropout apply on Attention output weights     
        #                                                      #
        AddNorm_out_1 = self.Norm1(Attn_output,Pos_Out)        #            Feed the input and attention output in Add & Norm Layer
        #                                                      #
        feedfwd = self.feedforward(AddNorm_out_1)              #            Feed the Add & Norm output in FeedForward Network
        #                                                      #
        fedfwd = self.Dropout2(feedfwd)                        #            Apply Dropout on FeedForward Weights
        #                                                      #
        AddNorm_out_2 = self.Norm2(feedfwd, AddNorm_out_1)     #       feed the feedforward and Add & Norm output again in Add & Norm Layer
        
        return AddNorm_out_2

In [35]:
enc_layer = EncoderLayer(dff=dff,dmodel=dmodel,num_heads=num_heads)

In [36]:
enc_output = enc_layer(Pos_out)

In [37]:
class Encoder(keras.layers.Layer):
    def __init__(self, num_layers, dmodel, dff, num_heads, maxlen, dropout_rate=0.1):
        super(Encoder,self).__init__()
        self.num_layers = num_layers
        self.dmodel = dmodel
        self.maxlen = maxlen
        #
        self.encoder = [EncoderLayer(dff=dff,dmodel=dmodel,num_heads=num_heads, dropout_rate=0.1) for _ in range(num_layers)]
        self.Positional_Encoding = PositionalEncoding(maxlen=maxlen, dmodel=dmodel)
    def call(self, X, Mask=None):
        for i in range(self.num_layers):
            X = self.encoder[i](X,Mask)
        return X

In [38]:
enc = Encoder(num_layers=num_layers, dmodel=dmodel, dff=dff, maxlen=maxlen, num_heads=num_heads)

In [39]:
Encoder_out = enc(Pos_out)

/home/greenfin/anaconda3/lib/python3.12/site-packages/keras/src/layers/layer.py:395: UserWarning: `build()` was called on layer 'encoder', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


In [40]:
Encoder_out

<tf.Tensor: shape=(9, 7, 16), dtype=float32, numpy=
array([[[-0.10379984, -2.1012635 ,  0.8208474 , ...,  0.778967  ,
          0.45996848, -0.3521647 ],
        [ 0.38393414, -2.2425632 ,  0.9678882 , ...,  0.6742509 ,
          0.37368912, -0.28314865],
        [ 0.43328968, -2.5287383 ,  1.1120346 , ...,  0.4716818 ,
          0.30702606, -0.25679955],
        ...,
        [-0.18311398, -2.5634594 ,  1.1886237 , ...,  0.42905447,
          0.32235393, -0.2831724 ],
        [-0.30466592, -2.2897666 ,  1.2193433 , ...,  0.5075541 ,
          0.37925676, -0.3391907 ],
        [-0.06462131, -2.0245876 ,  1.1932412 , ...,  0.546253  ,
          0.38940609, -0.36088   ]],

       [[-0.11003741, -2.1049519 ,  0.8393283 , ...,  0.7725291 ,
          0.4030059 , -0.35362634],
        [ 0.40204403, -2.215105  ,  0.95031524, ...,  0.68505394,
          0.33964005, -0.3159228 ],
        [ 0.43510303, -2.513033  ,  1.117787  , ...,  0.5154072 ,
          0.2498849 , -0.27126068],
        ...,
  

In [52]:
class DecoderLayer(keras.layers.Layer):
    def __init__(self, dmodel, dff, num_heads, dropout_rate = 0.1):
        super(DecoderLayer,self).__init__()

        self.MHA1 = MultiHeadAttention(num_heads=num_heads, dmodel=dmodel)
        self.MHA2 = MultiHeadAttention(num_heads=num_heads, dmodel=dmodel)
        
        self.Norm1 = AddNorm()
        self.Norm2 = AddNorm()
        self.Norm3 = AddNorm()
        #
        self.feedforward = FeedForwardNetwork(dmodel=dmodel, dff=dff)
        #
        self.Dropout1 = keras.layers.Dropout(dropout_rate)
        self.Dropout2 = keras.layers.Dropout(dropout_rate)
        self.Dropout3 = keras.layers.Dropout(dropout_rate)
        
    def call(self, decoder_input, encoder_output, look_ahead_mask=None, padding_mask=None):
        #First block in Decoder
        Attn_output1, _ = self.MHA1(decoder_input, decoder_input, decoder_input, look_ahead_mask)
        Attn_output1 = self.Dropout1(Attn_output1)
        AddNorm_out_1 = self.Norm1(Attn_output1,decoder_input)
        #
        Attn_output2, _ = self.MHA2(AddNorm_out_1, encoder_output, encoder_output, padding_mask)
        Attn_output2 = self.Dropout1(Attn_output2)
        AddNorm_out_2 = self.Norm2(Attn_output2,AddNorm_out_1)
        #
        feedfrwd = self.feedforward(AddNorm_out_2)
        feedfrwd = self.Dropout1(feedfrwd)
        AddNorm_out_3 = self.Norm3(feedfrwd, AddNorm_out_2)
        return AddNorm_out_3

In [53]:
dec = DecoderLayer(dmodel=dmodel, dff=dff, num_heads=num_heads)

In [54]:
dec_out = dec(decoder_input=Pos_out, encoder_output=Encoder_out)

In [55]:
dec_out

<tf.Tensor: shape=(9, 7, 16), dtype=float32, numpy=
array([[[-1.1417565 ,  0.36097828, -0.47348928, ...,  0.25004905,
         -0.33075348, -0.18860707],
        [-0.01600604, -0.45292014, -0.18953496, ...,  0.33417496,
         -0.42324764, -0.17961293],
        [ 0.02702629, -1.7481358 ,  0.2900324 , ...,  0.2810607 ,
         -0.2937296 , -0.15780878],
        ...,
        [-1.5628532 , -1.7186186 ,  0.6894515 , ...,  0.33504793,
         -0.00309771, -0.05813355],
        [-1.9268771 , -0.6829459 ,  0.81302696, ...,  0.21570994,
         -0.00550126, -0.13529274],
        [-1.3893032 ,  0.08602078,  0.7377381 , ...,  0.14902842,
         -0.11777079, -0.26334608]],

       [[-1.1334789 ,  0.2775583 , -0.5223977 , ...,  0.18637876,
         -0.32417998, -0.1087844 ],
        [ 0.06317283, -0.3905175 , -0.2932431 , ...,  0.18118615,
         -0.43747786, -0.18528077],
        [ 0.11477698, -1.7874432 ,  0.22718678, ...,  0.3920332 ,
         -0.27720997, -0.11182588],
        ...,
  

In [63]:
class Decoder(keras.layers.Layer):
    def __init__(self, num_layers, dmodel, dff, num_heads, maxlen, dropout_rate=0.1):
        super(Decoder, self).__init__()
        self.num_layers = num_layers
        self.dmodel = dmodel
        self.maxlen = maxlen
        #self.num_layers = num_layers
        #self.num_heads = num_heads
        #self.maxlen = maxlen
        #
        self.encoder = [DecoderLayer(dff=dff, dmodel=dmodel,num_heads=num_heads, dropout_rate=0.1) for _ in range(num_layers)]
    #
    def call(self, inputs, encoder_outputs, look_ahead_mask=None, padding_mask=None):
        dec_input = inputs
        for i in range(self.num_layers):
            dec_input = self.encoder[i](decoder_input=dec_input, encoder_output=encoder_outputs)
        return dec_input

In [65]:
dec = Decoder(num_layers=num_layers, dmodel=dmodel, dff=dff, num_heads=num_heads, maxlen=maxlen)

In [67]:
dec_out = dec(inputs=Pos_out, encoder_outputs=Encoder_out)

In [68]:
dec_out

<tf.Tensor: shape=(9, 7, 16), dtype=float32, numpy=
array([[[-1.2728891 , -0.7927553 , -0.55099463, ..., -1.1109413 ,
         -0.30989072, -0.74765813],
        [-1.040289  , -0.9389232 , -0.46707013, ..., -1.1099212 ,
         -0.39894858, -0.7226259 ],
        [-0.9705853 , -1.1883807 , -0.36747038, ..., -1.1542633 ,
         -0.33764282, -0.7378284 ],
        ...,
        [-1.276207  , -1.1261755 , -0.35420042, ..., -1.1615363 ,
         -0.16327494, -0.7492605 ],
        [-1.321234  , -0.9550736 , -0.36116228, ..., -1.1763622 ,
         -0.18478993, -0.7268417 ],
        [-1.2206286 , -0.8520108 , -0.37875646, ..., -1.1896614 ,
         -0.25451523, -0.69759977]],

       [[-1.2388754 , -0.79501426, -0.5425841 , ..., -1.1399553 ,
         -0.3511632 , -0.74213237],
        [-0.9807998 , -0.9066404 , -0.4574598 , ..., -1.1579353 ,
         -0.44182584, -0.7406214 ],
        [-0.9161773 , -1.1972747 , -0.38061222, ..., -1.1422706 ,
         -0.38226286, -0.7359839 ],
        ...,
  

In [89]:
class Transformer_Out(keras.layers.Layer):
    def __init__(self, vocab_size):
        super(Transformer_Out, self).__init__()
        self.Dens = keras.layers.Dense(units=vocab_size)
    def call(self, Decoder_output):
        return self.Dens(Decoder_output)

In [90]:
out = Transformer_Out(vocab_size=vocab_size)

In [96]:
out = out(dec_out)

In [92]:
embedding_weights = embedding_layer.get_weights()[0]  # This is the word embedding matrix

In [93]:
embedding_weights

array([[ 0.01419434,  0.04794849,  0.02349133,  0.02952555, -0.00793296,
        -0.00027902,  0.04083781,  0.03188333,  0.02939177, -0.03431376,
         0.03802017, -0.00647093, -0.03811002, -0.04870173,  0.00471635,
         0.00758206],
       [-0.00424042, -0.04119848, -0.01805595, -0.04892407, -0.03183792,
         0.03438416, -0.0299144 , -0.00067184, -0.01977938, -0.04267449,
        -0.03225761,  0.00054904, -0.01173132,  0.04775662,  0.02265276,
         0.04632482],
       [ 0.0212105 , -0.04559892, -0.00935124,  0.01131253,  0.00106335,
         0.04167651,  0.04165337, -0.03219356, -0.02484428, -0.04130856,
        -0.00358199,  0.03462228,  0.03045801, -0.0282076 ,  0.02168741,
        -0.02895192],
       [-0.02594265, -0.04670627,  0.013793  , -0.00840842,  0.03523368,
        -0.0072593 , -0.04749887, -0.01034052, -0.03466568, -0.01223063,
        -0.04193783,  0.01479859,  0.0216097 , -0.00706627, -0.01871919,
        -0.00097572],
       [ 0.04217869, -0.03192763,  0

In [99]:
predicted_token_ids = tf.argmax(out, axis=-1)

In [100]:
detokenized_texts = tokenizer.sequences_to_texts(predicted_token_ids.numpy())  # Convert token IDs to text

In [101]:
print(detokenized_texts)

["i'm i'm i'm i'm i'm i'm i'm", "i'm i'm i'm i'm i'm i'm i'm", "i'm i'm i'm i'm i'm i'm i'm", "i'm i'm i'm i'm i'm i'm i'm", "i'm i'm i'm i'm i'm i'm i'm", "i'm i'm i'm i'm i'm i'm i'm", "i'm i'm i'm i'm i'm i'm i'm", "i'm i'm i'm i'm i'm i'm i'm", "i'm i'm i'm i'm i'm i'm i'm"]


In [ ]:
mul = 2*i_mod

In [ ]:
mul

In [ ]:
inn = mul/ np.float32(128)

In [ ]:
inn

In [ ]:
angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(128))

In [ ]:
angle_rads = p*angle_rates

In [ ]:
angle_rads[5]

In [ ]:
angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

In [ ]:
angle_rads[5]

In [ ]:
angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

In [ ]:
angle_rads[0]

In [ ]:
positional_encoding = tf.cast(angle_rads[np.newaxis, ...], dtype=tf.float32)

In [ ]:
positional_encoding[0]

In [ ]:
inputs = tf.random.uniform((100, 1256))

In [ ]:
inputs = keras.layers.Dense(128)(inputs)

In [ ]:
inputs

In [ ]:
positional_encoding[:, :tf.shape(inputs)[1], :]

In [ ]:
positional_emb = inputs + positional_encoding[:, :tf.shape(inputs)[1], :]

In [ ]:
positional_emb[0][0]